1. loop over each image file name in the loaded list from the s3 bucket
2. check the image is already in grayscale or black and white
3. if image is already from grayscale, keep it as it is
4. if image is color, convert it to grayscale
5. save the converted image from the same file name as a png file

Step 1 – Install required packages

In [1]:
#install boto3 and pillow 
!pip install boto3 pillow


Step 2 - Import libraries and load image names from S3

In [2]:
# import boto3 to access AWS S3
import boto3

# import UNSIGNED and Config to allow public S3 access without credentials
from botocore import UNSIGNED
from botocore.client import Config

# import os to work with directories and file paths
import os

# import io to work with in-memory byte streams
import io

# import Image from Pillow to handle image operations
from PIL import Image

# set the S3 bucket name
BUCKET = "amazon-bodym"

# set the prefix (folder path) inside the bucket where images are stored
PREFIX = "testA/mask/"

# create an S3 client with unsigned (public) access
s3 = boto3.client("s3", config=Config(signature_version=UNSIGNED))

# create an empty list to store image file names
image_mask = []

# create a paginator to list many objects in the S3 folder
paginator = s3.get_paginator("list_objects_v2")

# loop through each page of results from the paginator
for page in paginator.paginate(Bucket=BUCKET, Prefix=PREFIX):
    # get the list of objects on this page (or empty list if none)
    for obj in page.get("Contents", []):
        # get the full key (path + file name) of the object
        key = obj["Key"]
        # check if the key ends with a common image extension
        if key.lower().endswith((".png", ".jpg", ".jpeg", ".bmp", ".tif", ".tiff", ".webp")):
            # take only the file name (last part after '/')
            filename_only = key.rsplit("/", 1)[-1]
            # add the file name to the list
            image_mask.append(filename_only)

# remove duplicates and sort the list of image names
image_mask = sorted(set(image_mask))

# print how many image names were loaded
print(len(image_mask), "names loaded")


1684 names loaded


Step 3 - Create the output folder

In [3]:
# set the name of the output folder for processed images
output_folder = "test - front masked images"

# create the folder if it does not already exist
os.makedirs(output_folder, exist_ok=True)

# print the path of the output folder
print("Output folder:", output_folder)


Output folder: test - front masked images


Step 4 - Download, check color vs grayscale, convert if needed, and save as PNG

In [4]:
# loop over each image file name in the list
for filename in image_mask:
    # build the full S3 key by joining prefix and file name
    s3_key = f"{PREFIX}{filename}"
    
    # print the file being processed (optional)
    print("Processing:", s3_key)
    
    # get the image object from S3
    response = s3.get_object(Bucket=BUCKET, Key=s3_key)
    
    # read the image bytes from the response body
    img_bytes = response["Body"].read()
    
    # wrap the bytes in a BytesIO object so Pillow can open it
    img_stream = io.BytesIO(img_bytes)
    
    # open the image using Pillow
    img = Image.open(img_stream)
    
    # check if the image is already grayscale or black and white
    # "L" and "1" are common modes for grayscale / bilevel images
    if img.mode in ("L", "1"):
        # if the image is already grayscale, keep it as it is
        gray_img = img
    else:
        # if the image is color (e.g., "RGB", "RGBA"), convert it to grayscale
        gray_img = img.convert("L")
    
    # get the base name without the extension (e.g., "0001" from "0001.png")
    base_name = os.path.splitext(filename)[0]
    
    # create the new file name with .png extension
    new_filename = base_name + ".png"
    
    # build the full output path inside the output folder
    output_path = os.path.join(output_folder, new_filename)
    
    # save the grayscale image as a PNG file at the output path
    gray_img.save(output_path, format="PNG")
    
    # optional: print the saved file path
    print("Saved to:", output_path)

# print a final message when all images are done
print("All images processed and saved in:", output_folder)


Processing: testA/mask/00112158c308d48307e95b841f678df8.png
Saved to: test - front masked images/00112158c308d48307e95b841f678df8.png
Processing: testA/mask/0048f0d9a4d0ca697e31077bab7b2471.png
Saved to: test - front masked images/0048f0d9a4d0ca697e31077bab7b2471.png
Processing: testA/mask/004a5121131d22f1c1d9f63828442577.png
Saved to: test - front masked images/004a5121131d22f1c1d9f63828442577.png
Processing: testA/mask/004e3469daa569584c9d56200fb93374.png
Saved to: test - front masked images/004e3469daa569584c9d56200fb93374.png
Processing: testA/mask/00aefca158a6b773b8c466c9b715ac25.png
Saved to: test - front masked images/00aefca158a6b773b8c466c9b715ac25.png
Processing: testA/mask/00e11c860f7b081a01e9c7f04405c40f.png
Saved to: test - front masked images/00e11c860f7b081a01e9c7f04405c40f.png
Processing: testA/mask/00e25ccd07165abbd859a16bbc569282.png
Saved to: test - front masked images/00e25ccd07165abbd859a16bbc569282.png
Processing: testA/mask/00eb5c77f79af6aea1fcf21184c1bf52.png
Sa

Step 5 - Compare S3 image count and local folder image count

In [5]:
# import os to work with directory listings
import os

# count how many image names were loaded from S3
num_s3_images = len(image_mask)  # get length of the list of S3 image names

# print the number of images found in S3
print("Number of image names from S3:", num_s3_images)

# list all files in the output folder
output_files = os.listdir(output_folder)  # get all file names in the output folder

# keep only files that end with .png (processed images)
output_images = [f for f in output_files if f.lower().endswith(".png")]  # filter only png files

# count how many processed images are in the folder
num_output_images = len(output_images)  # get length of the list of output images

# print the number of images in the output folder
print("Number of processed images in folder:", num_output_images)

# compare the counts from S3 and local folder
if num_s3_images == num_output_images:  # check if both counts are equal
    print("✅ Counts match: all images processed.")  # print success message
else:
    print("❌ Counts DO NOT match!")  # print mismatch message
    
    # calculate the difference between S3 and local counts
    diff = num_s3_images - num_output_images  # positive means missing, negative means extra
    
    # if diff is positive, some images are missing in the local folder
    if diff > 0:
        print(diff, "images are missing in the output folder.")  # report missing images
    else:
        # if diff is negative, there are extra images in the local folder
        print(-diff, "extra images are in the output folder.")  # report extra images


Number of image names from S3: 1684
Number of processed images in folder: 1684
✅ Counts match: all images processed.


Step 6 - Check that file names match between S3 and local folder

In [6]:
# import os to work with file names and paths
import os

# create a set of base names (without extension) from S3 image list
s3_base_names = set(os.path.splitext(name)[0] for name in image_mask)

# list all files in the output folder
output_files = os.listdir(output_folder)

# keep only .png files from the output folder
output_images = [f for f in output_files if f.lower().endswith(".png")]

# create a set of base names (without extension) from local png files
local_base_names = set(os.path.splitext(name)[0] for name in output_images)

# print how many unique base names are in each set
print("Unique S3 base names:", len(s3_base_names))
print("Unique local base names:", len(local_base_names))

# check if both sets of base names are exactly the same
if s3_base_names == local_base_names:
    # if sets are equal, all file names match
    print("✅ File names match between S3 and local folder (ignoring extension).")
else:
    # if sets differ, there is some mismatch
    print("❌ File names do NOT match between S3 and local folder.")
    
    # find names present in S3 but missing locally
    missing_locally = s3_base_names - local_base_names
    # find names present locally but not in S3
    extra_locally = local_base_names - s3_base_names
    
    # if there are names missing in local folder, print them
    if missing_locally:
        print("\nNames in S3 but missing in local folder:")
        for name in sorted(missing_locally):
            print("  ", name)
    
    # if there are extra names in local folder, print them
    if extra_locally:
        print("\nNames in local folder but not in S3:")
        for name in sorted(extra_locally):
            print("  ", name)


Unique S3 base names: 1684
Unique local base names: 1684
✅ File names match between S3 and local folder (ignoring extension).
